In [1]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
quant_path = "./awq_model"
model_path = "/mnt/data/opt-6.7b"
quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM"}

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-16 16:37:57.457508: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-16 16:37:58.040007: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 16:37:58.040048: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 16:37:58.043889: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alrea

In [7]:
model = AutoAWQForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.34s/it]


## 第一种方式

In [3]:
model.quantize(tokenizer, quant_config=quant_config)

ConnectionError: Couldn't reach 'mit-han-lab/pile-val-backup' on the Hub (ConnectionError)

## 与transformers 兼容

In [4]:
from transformers import AutoConfig,AwqConfig

In [5]:
# 修改配置文件以使其与transformers集成兼容
quantization_config = AwqConfig(
    bits=quant_config["w_bit"],
    group_size=quant_config["q_group_size"],
    zero_point=quant_config["zero_point"],
    version=quant_config["version"].lower(),
).to_dict()

# 预训练的transformers模型存储在model属性中，我们需要传递一个字典
model.model.config.quantization_config = quantization_config

In [6]:
print(model.model.config.quantization_config)

{'quant_method': <QuantizationMethod.AWQ: 'awq'>, 'bits': 4, 'group_size': 128, 'zero_point': True, 'version': <AWQLinearVersion.GEMM: 'gemm'>, 'backend': <AwqBackendPackingMethod.AUTOAWQ: 'autoawq'>, 'fuse_max_seq_len': None, 'modules_to_fuse': None, 'do_fuse': False}


In [7]:
tokenizer.save_pretrained(quant_path) 

('./awq_model/tokenizer_config.json',
 './awq_model/special_tokens_map.json',
 './awq_model/vocab.json',
 './awq_model/merges.txt',
 './awq_model/added_tokens.json',
 './awq_model/tokenizer.json')

# 这里不知道为什么运行时（model.save_quantized(quant_path)），会出现kernel died的情况，求解答

In [ ]:
model.save_quantized(quant_path)


## 调用模型

In [2]:

model_path = "/mnt/data/opt-6.7b"
model = AutoAWQForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 2/2 [00:51<00:00, 25.56s/it]


In [3]:
def generate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to(0)

    out = model.generate(**inputs, max_new_tokens=64)
    return tokenizer.decode(out[0], skip_special_tokens=True)


In [ ]:
result = generate_text("Merry Christmas! I'm glad to")
print(result)